In [ ]:
import requests
import pandas as pd

df = pd.read_csv('azure_test.csv')
base_url = "https://prices.azure.com/api/retail/prices"
params = {
    '$filter': "serviceName eq 'Virtual Machines'",
    'currencyCode': 'USD'
}

items = []
url = base_url + "?$filter=" + params['$filter'].replace(" ", "%20") + "&currencyCode=" + params['currencyCode']
while url:
    resp = requests.get(url).json()
    items.extend(resp.get('Items', []))
    url = resp.get('NextPageLink')
prices = pd.DataFrame(items)
df['_Region_key']  = df['Region'].str.lower()
df['_VMSize_key'] = df['VMSize'].str.lower()
prices['_Region_key']  = prices['armRegionName'].str.lower()
prices['_VMSize_key'] = prices['armSkuName'].str.lower()

merged = df.merge(
    prices[['unitOfMeasure','retailPrice','_Region_key','_VMSize_key']],
    on=['_Region_key','_VMSize_key'],
    how='left'
)

merged = merged[merged['unitOfMeasure']=='1 Hour'].copy()

merged.rename(columns={'retailPrice':'Cost Per Hour'}, inplace=True)
merged.to_csv('azure_test_with.csv', index=False)

print(merged.head())


In [26]:
import pandas as pd

df = pd.read_csv("azure_test_with.csv")

cols = df.columns.tolist()
# swap the names in the list
cols[8], cols[10] = cols[10], cols[8]

# reindex the DataFrame
df = df[cols]

df.to_csv("final_test_azure_file.csv", index=False)
print("Columns reordered (headers moved with their data).")


Columns reordered (headers moved with their data).
